<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-To-Probabilistic-Graphical-Models----Practical-Session-1" data-toc-modified-id="Introduction-To-Probabilistic-Graphical-Models----Practical-Session-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction To Probabilistic Graphical Models -- Practical Session 1</a></span><ul class="toc-item"><li><span><a href="#Question-0" data-toc-modified-id="Question-0-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Question 0</a></span></li><li><span><a href="#Question-1" data-toc-modified-id="Question-1-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Question 1</a></span><ul class="toc-item"><li><span><a href="#1-Choose-the-appropriate-random-variables,-define-their-domains,-write-down-the-generative-model-and-draw-the-associated-directed-graphical-model." data-toc-modified-id="1-Choose-the-appropriate-random-variables,-define-their-domains,-write-down-the-generative-model-and-draw-the-associated-directed-graphical-model.-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>1 Choose the appropriate random variables, define their domains, write down the generative model and draw the associated directed graphical model.</a></span></li><li><span><a href="#3" data-toc-modified-id="3-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>3</a></span></li><li><span><a href="#4-Implement-a-program-that-simulates-this-scenario;-i.e.,-generates-realizations-from-the-movements-of-the-robot-and-the-associated-sensor-readings.-Simulate-a-scenario-for-$k-=-1...100$-with-$N-=-50$,-$\epsilon-=-0,3$,$-w-=-0.8$." data-toc-modified-id="4-Implement-a-program-that-simulates-this-scenario;-i.e.,-generates-realizations-from-the-movements-of-the-robot-and-the-associated-sensor-readings.-Simulate-a-scenario-for-$k-=-1...100$-with-$N-=-50$,-$\epsilon-=-0,3$,$-w-=-0.8$.-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>4 Implement a program that simulates this scenario; i.e., generates realizations from the movements of the robot and the associated sensor readings. Simulate a scenario for $k = 1...100$ with $N = 50$, $\epsilon = 0,3$,$ w = 0.8$.</a></span></li></ul></li></ul></li></ul></div>

# Introduction To Probabilistic Graphical Models -- Practical Session 1

In [34]:
import numpy as np
from scipy.linalg import circulant

## Question 0

In [26]:
def derive(X):
    l_ = np.max(X)
    return l_ + np.log(np.exp((X - l_)).sum())

In [25]:
data = np.array([-1234.,-1235.], dtype=np.float64)
derive(data)

-1233.6867383124818

## Question 1

### 1 Choose the appropriate random variables, define their domains, write down the generative model and draw the associated directed graphical model.
D'après le sujet, chaque possible position est un état de robot.On definie la $y_t$, la variable d'état au temps t.On peut donc definie la problabilité de transition comme ci-dessous:

$P(y_t = j | y_{t-1} = i) =  a_{i,j} = 
\begin{cases}
\epsilon  &  si\: j=i \\
1-\sigma  &  si\: j=i+1 \\
0 & autre cas\\
\end{cases}
$

La matrix des probabilités de transitions 

$A = [a_{i,j}]$

Comme la distribution du positions initial est uniforme. Le vecteur de probabilités initiales est donc sous forme ci-dessous

$\Pi ={\pi_i} $ où $i \in [1,N] $ avec $\pi_i = \frac{1}{N}$

La probabilités d'observation des positions est donc 
$ b_{ij} = p(y_{1:k} = j |x_{i:k} = i) = 
\begin{cases}
\frac{1}{N} + w(1-\frac{1}{N})  &  j = i \\
(1-w)(1-\frac{1}{N})  &  j\neq i \\
\end{cases}
$


$\begin{pmatrix}
1 & 1& 11\\
1 & 1& 11\\
\vdots  \\
1 & 1&  \dots & x\\
\end{pmatrix}
$

###  3 

### 4 Implement a program that simulates this scenario; i.e., generates realizations from the movements of the robot and the associated sensor readings. Simulate a scenario for $k = 1...100$ with $N = 50$, $\epsilon = 0,3$,$ w = 0.8$.

In [44]:
a = np.identity(50)
np.roll(a,-1, axis=1)

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [101]:
def gen_status(i, epsilon):
    return np.random.choice([k,k+i,0],p=[epsilon, 1.0-epsilon, 0.0])

    
def seq_gene(len_seq, N, epsilon, w):
    # Gene value init:
    x = np.zeros(len_seq, dtype=np.int8)
    y = np.zeros(len_seq, dtype=np.int8)
    x[0] = np.random.randint(1,N)
    
    for k in range(1,100):
        choice = [x[k-1], x[k-1]+1 if x[k-1]<50 else 1 ]
        x[k] = np.random.choice(choice,p=[epsilon, 1.0-epsilon])
    
    pos = np.zeros(N+1)
    pos[1:] = np.arange(N) 
    
    for k in range(0,100):
            y[k] =  np.random.choice( [x[k], np.random.randint(1,N+1)],p=[w, 1.0-w])
            print([x[k], np.random.randint(1,N+1)])
    return [x,y]
    
    
    
seq_gene(100,50,0.3,0.8)    

36
[36, 12]
[36, 2]
[37, 2]
[37, 32]
[38, 19]
[39, 26]
[40, 43]
[41, 34]
[42, 6]
[43, 26]
[44, 33]
[45, 12]
[45, 42]
[46, 26]
[47, 34]
[48, 28]
[48, 22]
[48, 35]
[49, 7]
[49, 39]
[50, 6]
[1, 8]
[1, 11]
[2, 46]
[3, 8]
[4, 2]
[4, 31]
[4, 41]
[5, 4]
[6, 26]
[7, 15]
[7, 50]
[7, 16]
[8, 28]
[8, 44]
[8, 33]
[9, 5]
[10, 6]
[10, 48]
[11, 37]
[12, 45]
[13, 16]
[13, 18]
[14, 21]
[15, 30]
[16, 42]
[17, 19]
[18, 33]
[18, 26]
[19, 11]
[19, 27]
[20, 9]
[21, 23]
[22, 17]
[23, 30]
[24, 49]
[25, 15]
[26, 1]
[27, 46]
[27, 7]
[27, 15]
[28, 5]
[29, 19]
[30, 42]
[31, 50]
[32, 11]
[33, 9]
[33, 22]
[34, 27]
[35, 9]
[36, 33]
[37, 8]
[38, 7]
[39, 17]
[40, 27]
[40, 4]
[41, 8]
[42, 40]
[42, 43]
[43, 42]
[43, 8]
[44, 2]
[45, 45]
[45, 25]
[46, 37]
[47, 39]
[48, 34]
[48, 40]
[49, 36]
[49, 12]
[50, 4]
[50, 41]
[1, 16]
[1, 3]
[2, 14]
[3, 13]
[4, 15]
[5, 1]
[6, 11]
[6, 43]


[array([36, 36, 37, 37, 38, 39, 40, 41, 42, 43, 44, 45, 45, 46, 47, 48, 48,
        48, 49, 49, 50,  1,  1,  2,  3,  4,  4,  4,  5,  6,  7,  7,  7,  8,
         8,  8,  9, 10, 10, 11, 12, 13, 13, 14, 15, 16, 17, 18, 18, 19, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 27, 27, 28, 29, 30, 31, 32, 33, 33,
        34, 35, 36, 37, 38, 39, 40, 40, 41, 42, 42, 43, 43, 44, 45, 45, 46,
        47, 48, 48, 49, 49, 50, 50,  1,  1,  2,  3,  4,  5,  6,  6],
       dtype=int8),
 array([43, 36,  6, 37, 38, 39, 40,  3, 42, 43, 44, 14, 45, 46, 21, 48, 11,
        48, 49, 49, 48,  1,  1, 32,  3,  4,  4, 29,  5,  6, 12, 31,  7,  8,
         8,  8,  9, 10, 46,  7, 12, 13, 13, 14, 15, 16, 17, 18, 18, 19, 18,
        20, 21, 22, 23, 24, 25, 26, 27, 27, 27, 28, 22, 30, 31, 32, 33, 33,
        35, 35, 36, 37, 21, 39, 40, 40, 41, 42,  1, 43, 43, 44, 45, 45, 46,
        47, 48, 48, 49, 38, 50, 47,  1, 31,  2,  3,  4,  5,  6,  9],
       dtype=int8)]

In [76]:
[2, np.random.randint(1,50)]

[2, 35]

In [88]:

50?20:330

SyntaxError: invalid syntax (<ipython-input-88-579bf150f2a1>, line 2)

In [95]:
[5,23 if 2<50 else 1] 

[5, 23]